In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#from tensorflow.python.client import device_lib
#print device_lib.list_local_devices()

In [2]:
import tensorflow as tf
import numpy as np
from pdrnn import triangular_pmf, build_pdrnn
from StochasticDilateNet import StochasticDilateNet
from scipy.stats import bernoulli
import hdf5storage as hdst
from time import clock

/home/logos/envs/prob_dropout/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
batch_size = 64
head = np.random.randint(1, 8, size=(10,batch_size,1))
body = np.full([499,batch_size,1], 8)
tail = np.full([11,batch_size,1], 9)
seq = np.concatenate((head, body, tail), axis=0)
hdst.savemat('/mnt/hdd1/kqian3/rl_struct/seq',{'seq':seq})

In [3]:
data = hdst.loadmat('/mnt/hdd1/kqian3/rl_struct/seq')
seq = data['seq']

In [6]:
tf.reset_default_graph()
n_layers = 4
hidden_structs = [10]*n_layers
init_params = [(1.0,0.5),(2.0,0.5),(4.0,0.5),(150.0,50.0)]
lambda_b = 0.9
model = StochasticDilateNet(hidden_structs,
                            init_params,
                            n_layers=n_layers,
                            n_classes=8,
                            input_dims=1,
                            cell_type="RNN")
# define session
sess_config = tf.ConfigProto(allow_soft_placement=True,
                             log_device_placement=False,
                             intra_op_parallelism_threads=20,
                             inter_op_parallelism_threads=4)
sess_config.gpu_options.allow_growth = True

In [ ]:
model.struct_param

In [15]:
try:
    sess.close()
except:
    pass
    
sess = tf.Session(config=sess_config)
# initialize all the parameters
sess.run(tf.global_variables_initializer())

start = clock()
for step in range(500):
    
    feed_dict = {model.inputs: seq,
                 model.labels: np.transpose(seq[0:10,:,0],[1,0]),
                 model.rates[-1]: 85}
    _, loss_value, rates, accuracy = sess.run([model.weights_train_op, model.loss_for_w, model.rates, model.accuracy], 
                                   feed_dict=feed_dict)
    loss_value = sess.run(model.loss_for_w, feed_dict=feed_dict)
    assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
    
    if step % 10 == 0:
        model.saver_for_w.save(sess, "{}model".format('/mnt/hdd1/kqian3/rl_struct/'), global_step=step)
        print('Step {}, rates = {}, loss = {}, accuracy = {}'.format(step, rates, loss_value, accuracy))
print clock()-start        

Step 0, rates = [1, 2, 4, array(85, dtype=int32)], loss = 2.10063195229, accuracy = 0.125
Step 10, rates = [1, 2, 4, array(85, dtype=int32)], loss = 2.04335236549, accuracy = 0.13750000298
Step 20, rates = [1, 2, 4, array(85, dtype=int32)], loss = 2.01467895508, accuracy = 0.12187500298
Step 30, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.99414610863, accuracy = 0.13750000298
Step 40, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.9794151783, accuracy = 0.13750000298
Step 50, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.96907830238, accuracy = 0.18125000596
Step 60, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.96205496788, accuracy = 0.18125000596
Step 70, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.95669436455, accuracy = 0.18125000596
Step 80, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.95083618164, accuracy = 0.18125000596
Step 90, rates = [1, 2, 4, array(85, dtype=int32)], loss = 1.93932628632, accuracy = 0.18125000596
Step 100, rates = [1

In [ ]:
loss.append(1.89209055901)

In [ ]:
#sess.close()
feed_dict = {model.inputs: seq,
                 model.labels: np.transpose(seq[0:10,:,0],[1,0])}
print sess.run(model.loss_per_example, feed_dict=feed_dict)

In [ ]:
with tf.Session(config=sess_config) as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    model.saver_for_w.restore(sess, "{}model-100".format('/mnt/hdd1/kqian3/rl_struct/'))
    
    
    for step in range(1000):
        #seq = bernoulli.rvs(0.5, size=(100,32,1))
        feed_dict = {model.inputs: seq,
                     model.labels: seq[0,:,0]}     
        
        if np.mod(step, 10) < 5:
            _, loss_value = sess.run([model.weights_train_op, model.loss_for_w], 
                                     feed_dict=feed_dict)
        else:
            sess.run(model.struct_train_op, feed_dict=feed_dict)
        
        
        if step == 0:
            sess.run(tf.assign(model.b, loss_value))
        else:
            sess.run(tf.assign(model.b, (lambda_b*model.b+(1-lambda_b)*loss_value)))
        
        if step % 10 == 0:
            struct_params, rates, b, accuracy = \
            sess.run([model.struct_vars, model.rates, model.b, model.accuracy],
                    feed_dict=feed_dict)
            print('Step {}, struct params {},\n Rates {}, b = {}, accuracy = {}'.format(
                step, struct_params, rates, b, accuracy))
            
            #sess.run(model.struct_clip_op, feed_dict={model.inputs: seq})
            #struct_params = sess.run(model.struct_vars)
            #
            #print('After clip, struct params {}'.format(struct_params))

In [ ]:
with tf.Session(config=sess_config) as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    #model.saver_for_w.restore(sess, "{}model-100".format('/mnt/hdd1/kqian3/rl_struct/'))
    
    num_w_op = 400
    feed_dict_pi = {model.inputs: seq,
                    model.labels: np.transpose(seq[0:10,:,0],[1,0]),}
    for step in range(20000):
        #seq = bernoulli.rvs(0.5, size=(100,32,1))        
        
        if step % num_w_op == 0:
            if step >= 2*num_w_op:
                sess.run(model.struct_train_op, feed_dict=feed_dict_w)
                sess.run(model.struct_clip_op, feed_dict=feed_dict_w)
            
            if step != 0:
                loss_ = sess.run(model.loss_for_w, feed_dict=feed_dict_w)
                if step == num_w_op:
                    sess.run(tf.assign(model.b, loss_))
                else:
                    sess.run(tf.assign(model.b, (lambda_b*model.b+(1-lambda_b)*loss_)))
                
            rates = sess.run(model.rates, feed_dict=feed_dict_pi)
            
        feed_dict_w = {model.inputs: seq,
                       model.labels: np.transpose(seq[0:10,:,0],[1,0]),
                       model.rates[-1]: rates[-1]}    
            
        sess.run(model.weights_train_op, feed_dict=feed_dict_w)
        
        if step % num_w_op == num_w_op-1:
            struct_params, b, loss_value, accuracy = \
            sess.run([model.struct_vars, model.b, model.loss_for_w, model.accuracy],
                    feed_dict=feed_dict_w)
            print('Step {}, struct params {},\n Rates {}, b = {}, loss = {}, accuracy = {}'.format(
                step, struct_params, rates, b, loss_value, accuracy))
            
            #sess.run(model.struct_clip_op, feed_dict={model.inputs: seq})
            #struct_params = sess.run(model.struct_vars)
            #
            #print('After clip, struct params {}'.format(struct_params))

Step 399, struct params [(1.0, 0.5), (2.0, 0.5), (4.0, 0.5), (150.0, 50.0)],
 Rates [1, 2, 4, 128], b = 0.0, loss = 1.88026881218, accuracy = 0.22812500596
Step 799, struct params [(1.0, 0.5), (2.0, 0.5), (4.0, 0.5), (150.0, 50.0)],
 Rates [1, 2, 4, 171], b = 1.88026881218, loss = 1.81796133518, accuracy = 0.21406249702
Step 1199, struct params [(1.0, 1.0), (2.0, 1.0), (4.0, 1.0), (150.06873, 49.988342)],
 Rates [1, 2, 4, 168], b = 1.87403798103, loss = 0.646810114384, accuracy = 0.746874988079
Step 1599, struct params [(1.0, 1.0), (2.0, 1.0), (4.0, 1.0), (151.27052, 49.646088)],
 Rates [1, 2, 4, 126], b = 1.75131523609, loss = 0.35119253397, accuracy = 0.879687488079
Step 1999, struct params [(1.0, 1.0), (2.0, 1.0), (4.0, 1.0), (149.51295, 49.68403)],
 Rates [1, 2, 4, 163], b = 1.61130297184, loss = 1.8165050745, accuracy = 0.25156250596
Step 2399, struct params [(1.0, 1.0), (2.0, 1.0), (4.0, 1.0), (149.33778, 49.76759)],
 Rates [1, 2, 4, 137], b = 1.63182318211, loss = 1.93762934208,

In [ ]:
#loss = []
for rate in range(0,525,5):
    if rate == 0: rate = 1
    with tf.Session(config=sess_config) as sess:
        # initialize all the parameters
        sess.run(tf.global_variables_initializer())
        
        for step in range(500):
            feed_dict = {model.inputs: seq,
                         model.labels: np.transpose(seq[0:10,:,0],[1,0]),
                         model.rates[-1]: rate}
            sess.run(model.weights_train_op, feed_dict=feed_dict)
                    
        loss_value = sess.run(model.loss_for_w, feed_dict=feed_dict)
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
        #loss.append(loss_value)
        print 'Rate = {}, loss = {}'.format(rate, loss_value)    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(loss))
plt.show()

In [ ]:
hdst.savemat('/mnt/hdd1/kqian3/rl_struct/loss1_1',{'loss':np.array(loss)})

In [ ]:
print feed_dict_w[model.rates[-1]]
feed_dict_w[model.b] = 1.3
with tf.Session() as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    model.saver_for_w.restore(sess, "{}model-100".format('/mnt/hdd1/kqian3/rl_struct/'))
    
    print sess.run(model.loss_per_example, feed_dict = feed_dict_w)
    print sess.run(model.b, feed_dict = feed_dict_w)
    print sess.run(model.struct_param, feed_dict = feed_dict_w)
    
    print sess.run(tf.gradients(model.loss_for_pi, model.struct_param), feed_dict = feed_dict_w)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.gradients(model.loss_for_pi, model.struct_vars[1]), feed_dict = feed_dict))

In [ ]:
print model.sess.run(model.loss_per_example, feed_dict=feed_dict)

In [ ]:
# build the network
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, shape = [None, None, 1], name = 'inputs')

logits, structs, rates = build_pdrnn(x=inputs,
                                     hidden_structs=[4]*3,
                                     init_params=[(2.0,2.0),(2.0,2.0),(2.0,2.0)],
                                     n_layers=3,
                                     n_steps=20,
                                     n_classes=2,
                                     input_dims=1,
                                     cell_type="RNN")

In [ ]:
a = tf.ones([10, 16])
b = tf.split(a, [1]*10)

print(b)

In [ ]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 3]])
with tf.Session():
    print tf.pad(t, paddings, constant_values=0.0).eval() 

In [ ]:
tf.reset_default_graph()
init_params=[(1.0,0.5),(2.0,2.0)]
dilations = []
for l in xrange(2):
    with tf.variable_scope('struct_layer_{}'.format(l)):
        mu = tf.get_variable('mu', initializer=tf.constant(init_params[l][0]))
        sigma = tf.get_variable('sigma', initializer=tf.constant(init_params[l][1]))
                
    rates = tf.range(tf.ceil(tf.maximum(mu-sigma,0.5)), 
                     tf.ceil(tf.minimum(mu+sigma,20)))
    probs = triangular_pmf(rates, mu, sigma, 20)
    probs = tf.expand_dims(probs, 0)
    idx = tf.multinomial(tf.log(probs), 1)
    rates = tf.cast(rates, tf.int32)
    dilations.append(rates[idx[0][0]])  

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(probs)

In [ ]:
a = tf.constant([1])
b = a;
c = b;

with tf.Session() as sess:
    print sess.run(c, feed_dict = {b: [2]})

In [ ]:
from pdrnn import drnn_classification, _contruct_cells
from pdrnn import dRNN, multi_dRNN_with_dilations
# build the network
tf.reset_default_graph()

dilations = []
for l in xrange(3):
    d = tf.placeholder_with_default(tf.constant(l+1), [])
    dilations.append(d)

inputs = tf.placeholder(tf.float32, shape = [None, None, 1], name = 'inputs')
hidden_structs=[4]*3

# construct a list of cells
cells = _contruct_cells(hidden_structs=hidden_structs, cell_type='LSTM')

#logits = multi_dRNN_with_dilations(cells, inputs, hidden_structs, dilations)
#logits = dRNN(cells[0], inputs, dilations[0], 1, scope='default')

logits = drnn_classification(x=inputs,
                               hidden_structs=hidden_structs,
                               dilations=dilations,
                               n_classes=2,
                               input_dims=1,
                               cell_type='RNN')


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pred = sess.run(logits, feed_dict={inputs: np.random.uniform(size=[100,32,1])})
    print pred.shape